# SageMaker processing を利用して前処理を行う(処理時間10分弱程度)

## 処理概要

1. 前処理用コンテナの作成(詳細は[こちら](./container/Dockerfile))
2. 前処理(詳細は[こちら](./container/preprocess_code/preprocess.py))をキック
    1. S3 から前処理用コンテナに先程作成した zip ファイルをダウンロード(自動)
    2. zip ファイルを解凍
    3. 画像を開いてヒストグラム平坦化
    4. numpy array に格納
    5. npy ファイルを出力
    6. 前処理用コンテナから S3 にデータをアップロード(自動)
3. アップロードしたデータの URI を設定ファイルに書き込み

![](media/1_preprocess.png)

In [ ]:
# notebook のセルの横方向の表示範囲を広げる
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

## 使用するライブラリ等の読み込みと設定ファイルの読み込み

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
import yaml
role = get_execution_role()
with open('./setting.yaml', 'r') as yml:
    config = yaml.load(yml)
name = config['name']
zip_dataset_s3_uri = config['zip_dataset_s3_uri']
timestamp = config['timestamp']
print(f'role: {role}')
print(f'name: {name}')
print(f'zip_dataset_s3_uri: {zip_dataset_s3_uri}')
print(f'timestamp: {timestamp}')

## 前処理用コンテナの作成
* SageMaker ではビルトインコンテナとして [Apache Spark](https://docs.aws.amazon.com/sagemaker/latest/dg/use-spark-processing-container.html) と [scikit-learn](https://docs.aws.amazon.com/sagemaker/latest/dg/use-scikit-learn-processing-container.html) のコンテナがあるが、画像処理を扱うためのコンテナがない(scikit-image, opencv, pillow など)
* bring your own container することは可能なので、Image を[作成](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html)する
* SageMaker Notebook では Docker がプリインストールされているのでそのまま利用する
    1. ローカルで Image をビルドする
    2. Elastic Container Registry の リポジトリに Image を push する
    3. push した Image を利用して前処理を行う

In [ ]:
!cat ./container/Dockerfile

In [ ]:
# Image のビルド
%cd container
!docker build -t sagemaker-tf-handson-{name}-{timestamp} .
%cd ../

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = f'sagemaker-tf-handson-{name}-{timestamp}'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'コンテナは {image_uri} へ登録されています。')

## Processor インスタンス作成
前処理を行うためのプロセッサーインスタンスを作成して、ジョブを開始する

In [ ]:
processing_input_dir = '/opt/ml/processing/input'
processing_output_train_dir = '/opt/ml/processing/train'
processing_output_test_dir = '/opt/ml/processing/test'
job_name = f'sagemaker-preprocess-handson-{name}'

In [ ]:
processor = ScriptProcessor(base_job_name=job_name,
                                   image_uri=image_uri,
                                   command=['python3'],
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge'
                                  )

## 前処理を開始する

#### オレオレ前処理スクリプトの防ぎ方 3 例
* docker コンテナにスクリプトを入れ込み、起動コードにはそのスクリプトを呼び出す形を取る
* S3 のパスを指定する
* run する前に git pull するシェルスクリプトなどを作る

In [ ]:
!pygmentize ./preprocess_script/preprocess.py

In [ ]:
# zip ファイルを解凍してヒストグラム平坦化してnpyファイルにまとめる前処理
processor.run(code='./preprocess_script/preprocess.py', # S3 の URI でも可
                     inputs=[ProcessingInput(source=zip_dataset_s3_uri,destination=processing_input_dir)],
                     outputs=[
                         ProcessingOutput(output_name='train',source=processing_output_train_dir),
                         ProcessingOutput(output_name='test',source=processing_output_test_dir)],
                      arguments=[
                          '--hist-flatten', 'True',
                          '--input-dir',processing_input_dir,
                          '--output-train-dir',processing_output_train_dir,
                          '--output-test-dir',processing_output_test_dir
                      ]
                    )

In [ ]:
# ジョブの詳細を確認(前処理結果の格納先がわかる)
processor_description = processor.jobs[-1].describe()
print(processor_description)

In [ ]:
train_data_uri = processor_description['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
test_data_uri = processor_description['ProcessingOutputConfig']['Outputs'][1]['S3Output']['S3Uri']
print(f'train_data_uri: {train_data_uri}')
print(f'test_data_uri: {test_data_uri}')

In [ ]:
# 前処理結果の格納先を設定ファイルに書き込んで次の処理に連携
with open("./setting.yaml", mode='a') as f:
    f.write('train_data_uri: ' + train_data_uri +'\n')
    f.write('test_data_uri: ' + test_data_uri + '\n')